In [2]:
!pip install --use-deprecated=legacy-resolver tflite-model-maker
!pip install pycocotools
# !pip install -q opencv-python-headless==4.1.2.30

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import cv2


import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [4]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import cv2


import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [5]:
labels = {1:'apple', 2:'banana', 3:'orange'}

train_imgs_path = 'images/train'
train_anno_path = 'annotations/train'

test_imgs_path = 'images/test'
test_anno_path = 'annotations/test'

valid_imgs_path = 'images/test'
valid_anno_path = 'annotations/test'

train_data = object_detector.DataLoader.from_pascal_voc(images_dir=train_imgs_path, annotations_dir=train_anno_path, label_map=labels)
test_data = object_detector.DataLoader.from_pascal_voc(images_dir=test_imgs_path, annotations_dir=test_anno_path, label_map=labels)
# valid_data = object_detector.DataLoader.from_pascal_voc(images_dir=valid_imgs_path, annotations_dir=valid_anno_path, label_map=labels)

In [6]:
spec = model_spec.get('efficientdet_lite0')
spec.uri = 'https://storage.googleapis.com/tfhub-modules/tensorflow/efficientdet/lite0/feature-vector/1.tar.gz'
spec.input_image_shape = [512, 512]

In [5]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, )
# model.summary()

Epoch 1/50
30/30 [==============================] - 33s 304ms/step - det_loss: 1.6446 - cls_loss: 1.0779 - box_loss: 0.0113 - reg_l2_loss: 0.0631 - loss: 1.7077 - learning_rate: 0.0090 - gradient_norm: 1.4770
Epoch 2/50
30/30 [==============================] - 9s 300ms/step - det_loss: 1.2278 - cls_loss: 0.8036 - box_loss: 0.0085 - reg_l2_loss: 0.0631 - loss: 1.2910 - learning_rate: 0.0100 - gradient_norm: 2.9001
Epoch 3/50
30/30 [==============================] - 9s 296ms/step - det_loss: 0.9495 - cls_loss: 0.5921 - box_loss: 0.0071 - reg_l2_loss: 0.0632 - loss: 1.0127 - learning_rate: 0.0099 - gradient_norm: 2.8794
Epoch 4/50
30/30 [==============================] - 9s 299ms/step - det_loss: 0.7737 - cls_loss: 0.4729 - box_loss: 0.0060 - reg_l2_loss: 0.0633 - loss: 0.8370 - learning_rate: 0.0099 - gradient_norm: 2.9749
Epoch 5/50
30/30 [==============================] - 9s 302ms/step - det_loss: 0.6761 - cls_loss: 0.4137 - box_loss: 0.0052 - reg_l2_loss: 0.0633 - loss: 0.7395 - learn

In [6]:
model.evaluate(test_data)

1/1 [==============================] - 9s 9s/step



{'AP': 0.6376676,
 'AP50': 0.92163396,
 'AP75': 0.7654883,
 'APs': -1.0,
 'APm': 0.6565594,
 'APl': 0.642207,
 'ARmax1': 0.43535715,
 'ARmax10': 0.7303968,
 'ARmax100': 0.73531747,
 'ARs': -1.0,
 'ARm': 0.725,
 'ARl': 0.73857987,
 'AP_/apple': 0.73353606,
 'AP_/banana': 0.52729154,
 'AP_/orange': 0.6521752}

In [6]:
config = QuantizationConfig.for_float16()
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

48/60 [=======================>......] - ETA: 1:01